# Correlations in Economy data

In [ ]:
# imports

import pandas as pd 
import collections as cTools
import itertools as iTools
import numpy as np
import plotly.graph_objects as go
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import linregress


# ROAD INFRASTRUCTURE EXPENDITURE

In [ ]:
# Read all data

investment = pd.read_csv(r'C:\Hari\Q5_books\143\Project\infrastructure\investment.csv')
maintenance = pd.read_csv(r'C:\Hari\Q5_books\143\Project\infrastructure\maintenance.csv')
goodsTransport = pd.read_csv(r'C:\Hari\Q5_books\143\Project\infrastructure\goodsTransport.csv')

In [ ]:
# Filter countries

g20Countries = [
    'Canada',
    'France',
    'India',
    'Italy',
    'Japan',
    'Mexico',
    'Korea',
    'Türkiye',
    'United Kingdom',
    'United States',
]

In [ ]:
# Road way data Investment manipulation

investSeries = investment[investment['VARIABLE']=='I-INV-RD']
investSeries = investSeries.drop(columns=['COUNTRY','Variable','Measure','MEASURE','YEAR','Unit Code','Unit','PowerCode Code','PowerCode'])
investSeries = investSeries.drop(columns = ['Reference Period Code','Reference Period','Flag Codes','Flags','VARIABLE'])
mask = investSeries['Country'].str.contains('Moldova') | investSeries['Country'].str.contains('Netherlands') | investSeries['Country'].str.contains('Albania')
investSeries = investSeries[~mask]
investSeries['Country'] = pd.Categorical(investSeries['Country'])
investSeries.set_index('Country')
investSeries = investSeries.pivot_table(index='Country', columns='Year', values='Value', aggfunc='first')
investSeries = investSeries.apply(pd.to_numeric, errors='coerce')
investSeries.fillna(method='bfill', inplace=True)
investSeries = investSeries.drop(index = 'Montenegro, Republic of')
investSeries = investSeries.iloc[:,::3]
g20InvestSeries = investSeries[investSeries.index.isin(g20Countries)]

print('###########-> g20 Investment Series <- ###############')
print(g20InvestSeries)


In [ ]:
# Road way Maintenance manipulation

goodsTransport = maintenance[maintenance['VARIABLE']=='I-MTN-RD']
maintenanceSeries = goodsTransport.drop(columns=['COUNTRY','Variable','Measure','MEASURE','YEAR','Unit Code','Unit','PowerCode Code','PowerCode'])
maintenanceSeries = maintenanceSeries.drop(columns = ['Reference Period Code','Reference Period','Flag Codes','Flags','VARIABLE'])
mask = maintenanceSeries['Country'].str.contains('Moldova') | maintenanceSeries['Country'].str.contains('Netherlands')
maintenanceSeries = maintenanceSeries[~mask]
maintenanceSeries['Country'] = pd.Categorical(maintenanceSeries['Country'])
maintenanceSeries.set_index('Country')
maintenanceSeries = maintenanceSeries.pivot_table(index='Country', columns='Year', values='Value', aggfunc='first')
maintenanceSeries = maintenanceSeries.apply(pd.to_numeric, errors='coerce')
maintenanceSeries.fillna(method='ffill', inplace=True)
maintenanceSeries = maintenanceSeries.iloc[:,::3]
g20MaintenanceSeries = maintenanceSeries[maintenanceSeries.index.isin(g20Countries)]

print('###########-> g20 maintenance Series <- ###############')
print(g20MaintenanceSeries)

In [ ]:
# Road way export manipulation
goodsTransportSeries = goodsTransport[goodsTransport['VARIABLE']=='T-GOODS-RD-TOT']
goodsTransportSeries = goodsTransportSeries.drop(columns=['COUNTRY','Variable','YEAR','Unit Code','Unit','PowerCode Code','PowerCode'])
goodsTransportSeries = goodsTransportSeries.drop(columns = ['Reference Period Code','Reference Period','Flag Codes','Flags','VARIABLE'])
goodsTransportSeries['Country'] = pd.Categorical(goodsTransportSeries['Country'])
goodsTransportSeries.set_index('Country')
goodsTransportSeries = goodsTransportSeries.pivot_table(index='Country', columns='Year', values='Value', aggfunc='first')
goodsTransportSeries = goodsTransportSeries.apply(pd.to_numeric, errors='coerce')
goodsTransportSeries.fillna(method='ffill', inplace=True)
goodsTransportSeries = goodsTransportSeries.iloc[:,::3]
goodsTransportSeries = goodsTransportSeries.drop(columns=[2021],axis=1)
g20GoodsTransportSeries = goodsTransportSeries[goodsTransportSeries.index.isin(g20Countries)]
print('###########-> g20 Transport load Series <- ###############')
print(g20GoodsTransportSeries)

In [ ]:
# Finding Correlation between investment and Trade
investTransportCorr = g20InvestSeries.corrwith(g20GoodsTransportSeries, axis=1).sort_values()

print('###########-> Correlation <- ###############')
print(investTransportCorr)

# Finding R squared value

r2Values = []
for i in range(len(g20InvestSeries)):
    slope, intercept, r_value, p_value, std_err = linregress(g20InvestSeries.iloc[i], g20GoodsTransportSeries.iloc[i])
    r2Values.append(r_value ** 2)

print('###########-> Correlation <- ###############')
print(r2Values)

In [ ]:
# See how maintenance correlates with investment

maintenanceInvestCorr = g20MaintenanceSeries.corrwith(g20InvestSeries, axis=1)
print('###########-> Correlation <- ###############')
print(maintenanceInvestCorr)

# see how maintenance correlates with transport

maintenanceTransportCorr = g20MaintenanceSeries.corrwith(g20GoodsTransportSeries, axis=1)
print('###########-> Correlation <- ###############')
print(maintenanceTransportCorr)


In [32]:
# Plots of Correlation investment and Trade transport volume
fig = go.Figure(data=go.Scatter(
    x=investTransportCorr.index,
    y=[1] * len(investTransportCorr), # Set y-axis to 1 to plot bubbles along the x-axis
    mode='markers',
    showlegend=False,
    marker=dict(
        size=abs(investTransportCorr)*8,  # Bubble size based on absolute value of correlation, scaled for display
        color=investTransportCorr,            # Bubble color based on correlation
        colorscale='Turbo',          # Choose a color scale (optional)
        sizemode='diameter',
        sizeref=0.1,
        opacity=0.8,
        reversescale=True,
        colorbar=dict(
            title='Correlation'       # Title of the colorbar
        ),
    )
))

# Update the layout with axis titles and chart title
fig.update_layout(
    xaxis_title='Country',
    yaxis_title='Y-axis',
    title='Bubble Plot Example'
)

# Find the index of the bubble with the highest and lowest correlation
highest_index = investTransportCorr.idxmax()
lowest_index = investTransportCorr.idxmin()

fig.add_trace(
    go.Scatter(
        x=[highest_index,lowest_index],
        y=[1, 1],
        text=['Highest','Lowest'],
        mode="text",
        textposition="top center",
        textfont=dict(color="white"),
        showlegend=False
    )
)



# Show the chart
fig.show()

